In [1]:
import numpy as np 
import pygame 
from datetime import datetime
# import sys
# sys.path.append(r'C:\Users\lenovo\Desktop\Artificial Inteligence\Jupyter projects\Time Dilation')
from functions import *
from light_signals import *
from collections import deque

pygame 2.4.0 (SDL 2.26.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
"""
Global animation variables

""" 

screen_width = 1500
screen_height = 800
FPS = 100     # Frames per second

# Initial distance between moving body and stationary observer
distance = 1300
# Velocity is measured as a percentage of the speed of light - [0, 1) 
velocity = 0.85

In [3]:
def main_animation(screen_width, screen_height,FPS):
    
    '''
    Initializations
    '''
    pygame.init()
    bg_color = (255, 255, 255)
    win = pygame.display.set_mode((screen_width, screen_height))
    clock = pygame.time.Clock()
    pygame.font.init()
    font_big = pygame.font.Font(None, 36)
    message_font = pygame.font.Font(None, 20)
    
    units =15 # Every unit is equal to 1 light second
    
    length = distance/units # Length of one unit in pixels
    
    # The animation layer responsible for the signals data
    background_surface, markers, screen_objects = create_background(screen_width, screen_height, bg_color,units, length)  
    LightSignalClass = LightSignal
    LightSignal.receiver_position = {k: v for k, v in markers[0].items()}
    
    for key, value in screen_objects.items():
        print(key, value)
        print()
    
    '''
    Local variables
    '''
    is_receiver_chron_started = False
    receiver_frames_count = 0 # Tracks the frames when the receiver starts his chronometer   
      
    
    displacement_per_frame = length/ FPS # The distance that light travels for 1 second
    
    frame = 0 
    start_angle = - np.pi/16 
    end_angle = - start_angle # Boundaries of the signals fronts
    
    signal_radius = 0
    
    gamma_fac = gamma_factor(velocity) # Shows how much slower the clock on the spaceship runs
    
    # The front of the first light signal emmited from the ship.
    # It will be used for the calculations of the spaceship position    
    first_light_front = [100, 500]
    space_ship_pos = first_light_front.copy()
    
      
    
    emitted_signals = deque([]) # Contains the emitted from spaceship singles objects
    
    emitted_signals.append(LightSignalClass.create_signal(space_ship_pos.copy(), '0'))
    
    # pygame.draw.circle(win, (0, 0, 0),first_light_front , radius=2)
    # rect = [100 - signal_radius, 500 - signal_radius, 2*signal_radius, 2*signal_radius]
    # pygame.draw.arc(win, (255, 0, 0), rect, start_angle, end_angle, 2)
    # pygame.draw.circle(background_surface, (255, 255, 0), space_ship_pos, radius = 2)
    
    readings_dict = {} # Readings of the receiver
    
    spaceship_tickings = 1
    messages_count = 0

    running = True
    
    while running:
        clock.tick(FPS)
        frame += 1
      
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                
        
        tick = pygame.time.get_ticks()
        win.fill(bg_color)
        text = font_big.render(f'Frame number {frame} - time {tick}', True, (0,0,0))
        
        space_ship_time = (frame * gamma_fac) / FPS
        
        stationary_time = frame / FPS
        
        space_ship_text = font_big.render(f'Spaceship clock {space_ship_time:.2f}', True, (0, 0, 0))
        stationary_text = font_big.render(f'Stationary clock {stationary_time:.2f}', True, (0, 0, 0))
        
        received_time_text = font_big.render('Received time', True, (0, 0, 0))
        received_message_text = font_big.render('Received message', True, (0, 0, 0))
        
        win.blit(background_surface, (0, 0))
        win.blit(text, (10, 10))
        win.blit(space_ship_text, (10, 80))
        win.blit(stationary_text, (10, 50))
        win.blit(received_time_text, (1000, 10))
        win.blit(received_message_text, (1250, 10))
                                           
        
        first_light_front = update_first_light_front(first_light_front, displacement_per_frame)
        
        # pygame.draw.circle(win, (0, 0, 0),first_light_front , radius=2)
        space_ship_pos[0] = first_light_front[0] * velocity
        
        # Remove the the first signal in the queueWhen is reaching the receiver and
        # get data from it.
        if emitted_signals and emitted_signals[0].is_receiver_reached:
            if not is_receiver_chron_started:
                is_receiver_chron_started = True
            chronometer_time = receiver_frames_count / FPS
            signal = emitted_signals.popleft()
            readings_dict = update_readings(readings_dict, len(readings_dict),chronometer_time, signal.message)
            
        if readings_dict:
            show_readings(readings_dict, font_big, win)
            
        # Active when the receiver`s clock is started
        if is_receiver_chron_started:
            
            chronometer_time = receiver_frames_count / FPS
            chronometer_time_text = font_big.render(f'Stationary chronometer {chronometer_time:.2f}', True, (0, 0, 0))
            receiver_frames_count += 1
            
            win.blit(chronometer_time_text, (10, 120))
            
            
        
        for signal in emitted_signals:
            signal.update_signal_position(displacement_per_frame)
            
            message_coords = update_message_position(end_angle, signal)
            message = message_font.render(signal.message, True, (128,128, 128))
            win.blit(message, message_coords)
  
            pygame.draw.arc(win, (255, 255, 0), signal.rect, start_angle, end_angle, 2)
            pygame.draw.circle(win, (227, 18, 233), signal.origin_position, radius = 2)
        

        pygame.draw.circle(win, (255, 0, 0), space_ship_pos, radius = 2)
        
        if space_ship_time >= spaceship_tickings:
            signal_point = pygame.draw.circle(background_surface, (255, 255, 0), space_ship_pos, radius = 2)
               
            # message = str(len(emmited_signals)-1)
            messages_count += 1
            emitted_signals.append(LightSignalClass.create_signal(space_ship_pos.copy(), str(messages_count)))
            
            spaceship_tickings += 1
        
        pygame.display.update()
        pygame.display.flip()
    # Exit
    pygame.quit()

        

In [4]:
main_animation(screen_width, screen_height,FPS)

x_line_start 100

vertical_line_start 640

vertical_line_end 660

x_line_end 1400

y_line 650

vertical_line_x [100.0, 186.66666666666669, 273.33333333333337, 360.0, 446.6666666666667, 533.3333333333334, 620.0, 706.6666666666667, 793.3333333333334, 880.0, 966.6666666666667, 1053.3333333333335, 1140.0, 1226.6666666666667, 1313.3333333333335, 1400.0]

numbers [[15, 95.0, 675], [14, 181.66666666666669, 675], [13, 268.33333333333337, 675], [12, 355.0, 675], [11, 441.6666666666667, 675], [10, 528.3333333333334, 675], [9, 615.0, 675], [8, 701.6666666666667, 675], [7, 788.3333333333334, 675], [6, 875.0, 675], [5, 961.6666666666667, 675], [4, 1048.3333333333335, 675], [3, 1135.0, 675], [2, 1221.6666666666667, 675], [1, 1308.3333333333335, 675], [0, 1395.0, 675]]

signals_start_positions []

spaceship_position []

